In [1]:
# Import findspark and initialize.
import findspark
findspark.init()

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()
print(spark.version)

3.5.2


In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"



In [4]:
# 2. Create a temporary view of the DataFrame.
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True)


In [5]:
# Show DataFrame
df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [6]:
# Import date time functions
from pyspark.sql.functions import year

# Show the year for the date column
df.select(year(df["date"])).show()

+----------+
|year(date)|
+----------+
|      2022|
|      2021|
|      2019|
|      2019|
|      2022|
|      2019|
|      2020|
|      2019|
|      2020|
|      2021|
|      2019|
|      2021|
|      2020|
|      2021|
|      2019|
|      2021|
|      2021|
|      2019|
|      2019|
|      2019|
+----------+
only showing top 20 rows



In [7]:
# Save the year as a new column
df = df.withColumn("year", year(df['date']))
df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|year|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|2022|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|2021|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|2019|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|2019|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|2022|
|5aa00529-0533-46b...|2019-01-30|      2

In [8]:
# Check the current schema in the Spark session
current_schema = spark.conf.get("spark.sql.defaultCatalog")
print("Current Schema: ", current_schema)

Current Schema:  spark_catalog


In [9]:
# Create a temporary view for the DataFrame
df.createOrReplaceTempView("home_sales")

In [10]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
price4_bed_query = """
SELECT year(date) AS year, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE bedrooms = 4
GROUP BY  year(date)
ORDER BY year(date)
"""

# Execute the query for 3-bedroom, 3-bathroom homes
price4_bed_query = spark.sql(price4_bed_query)
price4_bed_query.show()

+----+---------+
|year|avg_price|
+----+---------+
|2019| 300263.7|
|2020|298353.78|
|2021|301819.44|
|2022|296363.88|
+----+---------+



In [11]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?

price3_bed_3_Bath_sold_by_year_query = """
SELECT date_built, year(date) AS year, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3
GROUP BY date_built, year(date)
ORDER BY date_built
"""

# Execute the query for 3-bedroom, 3-bathroom homes
price3_bed_3_Bath_sold_by_year = spark.sql(price3_bed_3_Bath_sold_by_year_query)
price3_bed_3_Bath_sold_by_year.show()

+----------+----+---------+
|date_built|year|avg_price|
+----------+----+---------+
|      2010|2022|296405.79|
|      2010|2019|284458.42|
|      2010|2021|289023.65|
|      2010|2020|304246.88|
|      2011|2022|306552.15|
|      2011|2019|284564.26|
|      2011|2020|288642.39|
|      2011|2021| 292662.9|
|      2012|2022|300950.42|
|      2012|2020|283971.97|
|      2012|2019|296330.24|
|      2012|2021|297861.19|
|      2013|2019|284170.31|
|      2013|2020|290906.15|
|      2013|2022|312449.58|
|      2013|2021|307075.45|
|      2014|2021|296860.85|
|      2014|2022| 274183.0|
|      2014|2019|283766.07|
|      2014|2020|301938.78|
+----------+----+---------+
only showing top 20 rows



In [12]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?

Avg_Price_3_bed_3_Bath_2_Floor_2000sqft_query = """
SELECT date_built, year(date) AS year, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3
AND sqft_living >=2000 AND floors = 2
GROUP BY date_built, year(date)
ORDER BY date_built
"""

# Execute the query for 3-bedroom, 3-bathroom homes
Avg_Price_3_bed_3_Bath_2_Floor_2000sqft_query = spark.sql(Avg_Price_3_bed_3_Bath_2_Floor_2000sqft_query)
Avg_Price_3_bed_3_Bath_2_Floor_2000sqft_query.show()


+----------+----+---------+
|date_built|year|avg_price|
+----------+----+---------+
|      2010|2022|302258.94|
|      2010|2019|273667.58|
|      2010|2021|277498.27|
|      2010|2020| 298438.0|
|      2011|2022|290493.56|
|      2011|2019|270466.83|
|      2011|2020|256722.71|
|      2011|2021|298727.59|
|      2012|2022|305840.13|
|      2012|2020|324302.59|
|      2012|2019|304885.46|
|      2012|2021|300436.96|
|      2013|2019| 285635.4|
|      2013|2020|314235.11|
|      2013|2022|295597.11|
|      2013|2021|314629.69|
|      2014|2021|276998.95|
|      2014|2022|303618.52|
|      2014|2019|306384.98|
|      2014|2020|304561.62|
+----------+----+---------+
only showing top 20 rows



In [13]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating. 
# Although this is a small dataset, determine the run time for this query.
start_time = time.time()

Avg_Price_Rating_over350K_query = """
SELECT view, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
GROUP BY view
HAVING AVG(price) >= 350000
ORDER BY view DESC
"""

# Execute the query for 3-bedroom, 3-bathroom homes
Avg_Price_Rating_over350K_query = spark.sql(Avg_Price_Rating_over350K_query)
Avg_Price_Rating_over350K_query.show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view| avg_price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

--- 0.5215010643005371 seconds ---


In [14]:
# 7. Cache the the temporary table home_sales.
spark.catalog.cacheTable("home_sales")

In [15]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [16]:
# 9. Using the cached data, run the last query above, that calculates 
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000. 
# Determine the runtime and compare it to the uncached runtime.
start_time = time.time()

result = spark.sql("""
SELECT LPAD(view, 2, '0') AS Home_View_Rating, ROUND(AVG(price), 2) AS Avg_Price
FROM home_sales
WHERE price >= 350000
GROUP BY 1
ORDER BY Home_View_Rating DESC
""")

result.show()

print("--- %s seconds ---" % (time.time() - start_time))



+----------------+----------+
|Home_View_Rating| Avg_Price|
+----------------+----------+
|              99|1061201.42|
|              98|1053739.33|
|              97|1129040.15|
|              96|1017815.92|
|              95| 1054325.6|
|              94| 1033536.2|
|              93|1026006.06|
|              92| 970402.55|
|              91|1137372.73|
|              90|1062654.16|
|              89|1107839.15|
|              88|1031719.35|
|              87| 1072285.2|
|              86|1070444.25|
|              85|1056336.74|
|              84|1117233.13|
|              83|1033965.93|
|              82| 1063498.0|
|              81|1053472.79|
|              80| 991767.38|
+----------------+----------+
only showing top 20 rows

--- 1.0874059200286865 seconds ---


In [17]:
# 9. Using the cached data, run the last query above, that calculates 
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000. 
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()

Avg_Price_Rating_over350K_query = """
SELECT view, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
GROUP BY view
HAVING AVG(price) >= 350000
ORDER BY view DESC
"""

# Execute the query for 3-bedroom, 3-bathroom homes
Avg_Price_Rating_over350K_query = spark.sql(Avg_Price_Rating_over350K_query)
Avg_Price_Rating_over350K_query.show()


print("--- %s seconds ---" % (time.time() - start_time))


+----+----------+
|view| avg_price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

--- 0.29346418380737305 seconds ---


In [ ]:
# Determine the runtime and compare it to the uncached runtime.
print("the parquet runtime of 1.0874 seconds is slightly shorter than the cached runtime of 0.2934 seconds")

In [ ]:
#################################

In [19]:
parquet_df.write.parquet('home_parquet', mode='overwrite')

NameError: name 'parquet_df' is not defined

In [ ]:
##############################

In [14]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
#output_path = "path_to_output_directory"
#partition_column = "date_built"

#parquet_df = f"{output_path}/{partition_column}"

In [ ]:
# 11. Read the formatted parquet data.
parquet_df.write.partitionBy(partition_column).parquet(output_path)

In [ ]:
# 12. Create a temporary table for the parquet data.
parquet_df.createOrReplaceTempView("parquet_data_table")

In [ ]:
# 12. Create a temporary table for the parquet data.
parquet_df.createOrReplaceTempView("parquet_data_table")

In [23]:
# 13. Using the parquet DataFrame, run the last query above, that calculates 
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000. 
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()
# Run the query to calculate the average price of a home per "view" rating with average home price >= $350,000
result = spark.sql("""
    SELECT view, ROUND(AVG(price), 2) AS avg_price
    FROM home_sales
    GROUP BY view
    HAVING AVG(price) >= 350000
""")

result.show()


print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view| avg_price|
+----+----------+
|  85|1056336.74|
|  65| 736679.93|
|  53|  755214.8|
|  78|1080649.37|
|  81|1053472.79|
|  76|1058802.78|
|  91|1137372.73|
|  93|1026006.06|
|  52| 733780.26|
|  86|1070444.25|
|  94| 1033536.2|
|  57|  734340.5|
|  54| 798684.82|
|  96|1017815.92|
|  92| 970402.55|
|  64| 767036.67|
|  61| 746877.59|
|  88|1031719.35|
|  72| 780914.67|
|  59|  791453.0|
+----+----------+
only showing top 20 rows

--- 0.32926416397094727 seconds ---


In [ ]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("home_sales")

In [13]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached("home_sales")
